In [17]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta

pd.options.display.max_columns = 5000
pd.options.display.max_rows = 5000

In [50]:
# Load Dataset

# Renzo's Path
dataset = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/complete_final_df_3.csv')

# Sravan's Path
# dataset = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/complete_final_df_3.csv')

/Users/renzomaldonado/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [51]:
# Creating variable 'was_match' to know which rows are matches (real games) and which rows are not

dataset.loc[dataset['Min'].isnull(), 'was_match'] = 0
dataset.loc[dataset['Min'].isnull() == False, 'was_match'] = 1

In [52]:
# Filling in some missing values

dataset.loc[dataset['name'].isnull(), 'Prog'] = dataset['PlayerName']

# [Min, Gls, Ast, PK, PKatt, Sh, SoT, CrdY, CrdR, Touches, Press, Tkl, Int, Blocks, xG, npxG, xA, SCA, GCA, Prog, Carries, Prog.1, Succ, Att.1, \
# Fls, Off, Crs, TklW, OG, PKwon, PKcon, Won, Loss, Draw] all have 150755 NaN's, of which none are when matches are played, so we equal those \
# NaN's in these variables to 0
dataset.loc[dataset['Min'].isnull(), 'Min'] = 0
dataset.loc[dataset['Gls'].isnull(), 'Gls'] = 0
dataset.loc[dataset['Ast'].isnull(), 'Ast'] = 0
dataset.loc[dataset['PK'].isnull(), 'PK'] = 0
dataset.loc[dataset['PKatt'].isnull(), 'PKatt'] = 0
dataset.loc[dataset['Sh'].isnull(), 'Sh'] = 0
dataset.loc[dataset['SoT'].isnull(), 'SoT'] = 0
dataset.loc[dataset['CrdY'].isnull(), 'CrdY'] = 0
dataset.loc[dataset['CrdR'].isnull(), 'CrdR'] = 0
dataset.loc[dataset['Touches'].isnull(), 'Touches'] = 0
dataset.loc[dataset['Press'].isnull(), 'Press'] = 0
dataset.loc[dataset['Tkl'].isnull(), 'Tkl'] = 0
dataset.loc[dataset['Int'].isnull(), 'Int'] = 0
dataset.loc[dataset['Blocks'].isnull(), 'Blocks'] = 0
dataset.loc[dataset['xG'].isnull(), 'xG'] = 0
dataset.loc[dataset['npxG'].isnull(), 'npxG'] = 0
dataset.loc[dataset['xA'].isnull(), 'xA'] = 0
dataset.loc[dataset['SCA'].isnull(), 'SCA'] = 0
dataset.loc[dataset['GCA'].isnull(), 'GCA'] = 0
dataset.loc[dataset['Prog'].isnull(), 'Prog'] = 0
dataset.loc[dataset['Carries'].isnull(), 'Carries'] = 0
dataset.loc[dataset['Prog.1'].isnull(), 'Prog.1'] = 0
dataset.loc[dataset['Succ'].isnull(), 'Succ'] = 0
dataset.loc[dataset['Att.1'].isnull(), 'Att.1'] = 0
dataset.loc[dataset['Fls'].isnull(), 'Fls'] = 0
dataset.loc[dataset['Off'].isnull(), 'Off'] = 0
dataset.loc[dataset['Crs'].isnull(), 'Crs'] = 0
dataset.loc[dataset['TklW'].isnull(), 'TklW'] = 0
dataset.loc[dataset['OG'].isnull(), 'OG'] = 0
dataset.loc[dataset['PKwon'].isnull(), 'PKwon'] = 0
dataset.loc[dataset['PKcon'].isnull(), 'PKcon'] = 0
dataset.loc[dataset['Won'].isnull(), 'Won'] = 0
dataset.loc[dataset['Loss'].isnull(), 'Loss'] = 0
dataset.loc[dataset['Draw'].isnull(), 'Draw'] = 0

In [53]:
# Assigning Dummy Variables for player position from 'Position:'

dataset.loc[dataset['Position:'].isnull(), 'Position:'] = ''
dataset['defender'] = np.where(dataset['Position:'].str.contains('Defender'), 1, 0)
dataset['attacker'] = np.where(dataset['Position:'].str.contains('attack'), 1, 0)
dataset['midfielder'] = np.where(dataset['Position:'].str.contains('midfield'), 1, 0)
dataset['goalkeeper'] = np.where(dataset['Position:'].str.contains('Goalkeeper'), 1, 0)


In [54]:
# Creating the Column Age from current_date and birth and cleaning unneeded features

dates = (dataset['agg_year'] * 100 + dataset['agg_week']).astype(int)

dataset['date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')

dataset['age'] = round((pd.to_datetime(dataset['date']) - pd.to_datetime(dataset['Birth'])) / timedelta(days=365), 0)

dataset['retired'] = np.where(dataset['Retired since:'].isnull(), 0, 1)

dataset.drop(columns = ['Retired since:', 'Position:', 'PlayerName', 'FootAbility', 'Ban since:', 'current_year', 'current_week', 'FBRefId', 'date', 'Birth'], inplace = True)


dataset[dataset['FBRefID'] == '71672fa0'].tail(10)

,name,FBRefID,agg_week,agg_year,Injury,injury_week,injury_year,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,release_week,Height,Weight,Foot,was_match,defender,attacker,midfielder,goalkeeper,age,retired
517327,Petr-Cech,71672fa0,10.0,2019.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,35.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517328,Petr-Cech,71672fa0,11.0,2019.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,26.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517329,Petr-Cech,71672fa0,14.0,2019.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517330,Petr-Cech,71672fa0,15.0,2019.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,35.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517331,Petr-Cech,71672fa0,16.0,2019.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,48.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,45.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517332,Petr-Cech,71672fa0,17.0,2019.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517333,Petr-Cech,71672fa0,18.0,2019.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,22.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517334,Petr-Cech,71672fa0,19.0,2019.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,46.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517335,Petr-Cech,71672fa0,22.0,2019.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,25.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,37.0,1
517336,Petr-Cech,71672fa0,51.0,2020.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,39.0,1


In [55]:
# Sorting values in dataset_for_model

dataset = dataset.sort_values(['FBRefID', 'agg_year', 'agg_week'])

In [61]:
# Creating 'injured' column

dataset.loc[dataset['Injury'] != '0', 'injured'] = 1
dataset.loc[dataset['Injury'] == '0', 'injured'] = 0

dataset[dataset['FBRefID'] == '71672fa0'].tail(60)


,name,FBRefID,agg_week,agg_year,Injury,injury_week,injury_year,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,release_week,Height,Weight,Foot,was_match,defender,attacker,midfielder,goalkeeper,age,retired,injured
517277,Petr-Cech,71672fa0,46.0,2017.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,33.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517278,Petr-Cech,71672fa0,47.0,2017.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,26.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517279,Petr-Cech,71672fa0,48.0,2017.0,0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,56.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517280,Petr-Cech,71672fa0,49.0,2017.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,35.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517281,Petr-Cech,71672fa0,50.0,2017.0,0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,56.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517282,Petr-Cech,71672fa0,51.0,2017.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,42.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517283,Petr-Cech,71672fa0,52.0,2017.0,0,0.0,0.0,270.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,58.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517284,Petr-Cech,71672fa0,1.0,2018.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,29.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517285,Petr-Cech,71672fa0,2.0,2018.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,17.0,35.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0
517286,Petr-Cech,71672fa0,3.0,2018.0,0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,30.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,196.0,90.0,LEFT,1.0,0,0,0,0,36.0,1,0.0


In [46]:
len(dataset_for_model_final['FBRefID'].unique())

,FBRefId,Height,Weight,Foot,Birth
0,3bb7b8b4,188.0,89.0,LEFT,"August 17, 1993"
1,31c69ef1,186.0,82.0,RIGHT,"May 14, 1997"
2,6434f10d,190.0,78.0,RIGHT,"June 22, 1996"
3,b400bde0,170.0,70.0,RIGHT,"December 8, 1994"
4,bd6351cd,182.0,66.0,RIGHT,"May 27, 1994"


1523614.0

In [297]:
# dataset_for_model_final[dataset_for_model_final['FBRefID'] == '71672fa0']

In [298]:
# Saving dataset_for_model_final to Google Drive

# Renzo's path
dataset_for_model_final.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/dataset_for_model_final.csv', index=False)